In [1]:
# Just ignore this shit
import sys
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# starts here
hr_data = pd.read_excel("data-2.xlsx", sheet_name='HR Data', engine='openpyxl')
exit_data = pd.read_excel("data-2.xlsx", sheet_name='HR Exit', engine='openpyxl')
    
# Just printing the data, to visualize better
print ("HR Data size:", len(hr_data))
print ("Exit Data size:", len(exit_data))

HR Data size: 10098
Exit Data size: 1838


In [3]:
# How many voluntary terminated employees
voluntary_exit_data = exit_data.loc[exit_data['Name of action type'] == 'Voluntary Termination']
print("Voluntary terminations in exit HR:", len(voluntary_exit_data))

Voluntary terminations in exit HR: 996


In [4]:
# Let's make a new column and fill it in, 
# For exit data, if Voluntary Terminal == 1, else 0
exit_data['Voluntary Turnover'] = np.where(exit_data['Name of action type'] == 'Voluntary Termination', 1, 0)

# For Hr data, If the employee, is found in the HR exit list of voluntary terminated employees, 
# we update to 1, it otherwise ... 0 . that's what i'm going with 
hr_data['Voluntary Turnover'] = hr_data['EmployeeID'].isin(voluntary_exit_data['EmployeeID'].values).astype(int)

# Just printing the data, to visualize
print("Voluntary terminations in HR data:", len(hr_data.loc[hr_data['Voluntary Turnover'] == 1]))

Voluntary terminations in HR data: 767


In [5]:
# So let's make a new column and fill it in,

# For exit data, they're all "1" since they are all terminated I guess?, so all 1s
exit_data['All Turnover'] = 1

# For hr data, we can put it to 1 for all accounts that are found in the EXIT HR data
hr_data['All Turnover'] = hr_data['EmployeeID'].isin(exit_data['EmployeeID'].values).astype(int)

# Just printing the data, to visualize
# printed only the ones that are have turnover
print("All terminations in HR data:", len(hr_data.loc[hr_data['All Turnover'] == 1]))

All terminations in HR data: 1392


In [6]:
# Tenure date ...
# So every employee, from the date they started till date of report (in days)? .. aight
 
# Here we go through every column, and parse the string into a datetime object that we can do some nice operations on
# BTW: here we use errors='coerce' to force the function 'to_datetime' to kinda ignore invalid dates ish
hr_data['Original Hire Date'] = pd.to_datetime(hr_data['Original Hire Date'], errors='coerce')
exit_data['OrgHire'] = pd.to_datetime(exit_data['OrgHire'], errors='coerce')

# So where the hell is the date of the report?, but aight ill just make one
hr_data['Report date'] = pd.to_datetime('2014-01-15')
exit_data['Report date'] = pd.to_datetime('2014-01-15')

# Now we calculate the dates
hr_data['Firm Tenure days'] = (hr_data['Report date'] - hr_data['Original Hire Date']).dt.days
exit_data['Firm Tenure days'] = (exit_data['Report date'] - exit_data['OrgHire']).dt.days

# Just printing the reuslts to visualize
# Looks like we have some negative days in HR_exit data, some people got hired after the date Jan 15 2015,
# I mean if we just have a more recent report date should be fine
hr_data[['EmployeeID', 'Original Hire Date', 'Report date', 'Firm Tenure days']].tail()

,EmployeeID,Original Hire Date,Report date,Firm Tenure days
10093,8831,2012-02-13,2014-01-15,702.0
10094,9137,2012-08-13,2014-01-15,520.0
10095,3898,2012-08-13,2014-01-15,520.0
10096,1929,2012-09-10,2014-01-15,492.0
10097,9576,2012-10-18,2014-01-15,454.0


In [7]:
# Job tenure date
# Very similar to above

# Again we go through every column, and parse the string into a datetime object that we can do some nice operations on
# BTW: we also include the error='coerce' option
hr_data['Current Hire Date'] = pd.to_datetime(hr_data['Current Hire Date'], errors='coerce')
exit_data['Cur.Hire'] = pd.to_datetime(exit_data['Cur.Hire'], errors='coerce')

# We already created the report date, so we don't need to re-create it ... 

# Now we calculate the dates
hr_data['Job Tenure days'] = (hr_data['Report date'] - hr_data['Current Hire Date']).dt.days
exit_data['Job Tenure days'] = (exit_data['Report date'] - exit_data['Cur.Hire']).dt.days

# And finally we print to visualize the results
# I guess results are similar, cuz those 2 fields are prety much always the same
hr_data[['EmployeeID', 'Current Hire Date', 'Report date', 'Job Tenure days']].tail()

,EmployeeID,Current Hire Date,Report date,Job Tenure days
10093,8831,2012-02-13,2014-01-15,702
10094,9137,2012-08-13,2014-01-15,520
10095,3898,2012-08-13,2014-01-15,520
10096,1929,2012-09-10,2014-01-15,492
10097,9576,2012-10-18,2014-01-15,454


In [8]:
# So I have no clue wtf is correlation between 2 columns, but we got something for that in the pandas library
# Guess i'll do it for both data sets
print ('Correlation:', hr_data['Firm Tenure days'].corr(hr_data['All Turnover']))
print ('Correlation:', exit_data['Firm Tenure days'].corr(hr_data['All Turnover']))

Correlation: 0.0037161996974683104
Correlation: -0.03322931759599934


In [9]:
# Pivot table
pd.pivot_table(hr_data, values='Voluntary Turnover', index=['PS text'], aggfunc={np.mean})

,mean
PS text,
Bus Development,0.040000
Business Dev,0.000000
CHC Marketing,0.150000
CHC Nat,0.000000
CHC Sales,0.000000
Development,0.077895
Dist Warehouse,0.064516
Drug Reg Affair,0.018868
Engineering,0.071584


In [10]:
# I guess that's it! I'm pretty sure not everything is correct, but should be close enough i guess!

In [11]:
# Download data frames for verifications
hr_data.to_csv('HR_data.csv', index = False)
exit_data.to_csv('Exit_data.csv', index = False)